In [2]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
from datasets import Dataset
from tqdm import tqdm

In [3]:
# Load the test set
with open('../data/test_TLQA.json', 'r') as f:
    test_data = json.load(f)

# Load the train set
with open('../data/train_TLQA.json', 'r') as f:
    train_data = json.load(f)

# Load the contexts
with open('../data/preprocessed_contexts.json', 'r') as f:
    context_data = json.load(f)

# Get the necessary details from the input
def prepare_dataset(data):
    questions, answers = [], []
    for item in data:
        question = item['question']
        answer = '; '.join(item['final_answers'])
        questions.append(question)
        answers.append(answer)
    return {"question": questions, "answer": answers}

test_dataset_dict = prepare_dataset(test_data)
test_dataset = Dataset.from_dict(test_dataset_dict)

train_dataset_dict = prepare_dataset(train_data)
train_dataset = Dataset.from_dict(train_dataset_dict)

# Create a new DataFrame
df = pd.DataFrame()
df["question"] = test_dataset_dict["question"]
df["answer"] = test_dataset_dict["answer"]
df["context"] = df["question"].map(context_data)

In [4]:
from components.knn import KNN

k_value = 3
knn_classifier = KNN(k=k_value)

# Train the knn classifier on the train data
knn_classifier.fit(train_dataset_dict["question"], train_dataset_dict["answer"])

In [ ]:
# Create a summarization pipeline for the context
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)

# Create a tokenizer for the context
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
model = "google/flan-t5-xl"
pipe = pipeline(
    "text2text-generation",
    model=model, max_new_tokens=415,
    tokenizer=AutoTokenizer.from_pretrained(model),
    do_sample=True,
    temperature=1e-5,
    top_p=0.9,
    device=0
)

results = []

max_length_context = 90

k_context = 10

# Prompt for each instance in the test set
for i, prompt_question in enumerate(tqdm(df["question"], desc="Processing Questions")):
    # Select few shot examples using the knn classifier
    closest_prompts = knn_classifier.query(prompt_question)
    few_shot_examples = examples = "\n".join(
        f"Example:\n  Question: {closest_prompts_question}\n  Answer: {closest_prompts_answers}" for closest_prompts_question, closest_prompts_answers in closest_prompts
    )

    top_k_contexts = df["context"].iloc[i][:k_context]
    top_k_contexts = [ctx.replace("\n", " ") for ctx in top_k_contexts]

    # Summarize each context individually
    summarized_contexts = []
    for ctx in top_k_contexts:
        # Tokenize the context
        tokenized_context = tokenizer(ctx, return_tensors="pt", truncation=False)
        token_length = len(tokenized_context["input_ids"][0])

        # Summarize if the context is too long
        if token_length > max_length_context:
            summarized_content = summarizer(ctx, max_length=max_length_context, min_length=30, truncation=True)[0]["summary_text"]
            summarized_contexts.append(summarized_content)
        else:
            summarized_contexts.append(ctx)

    # Combine summarized contexts
    combined_summaries = " ".join(summarized_contexts)


    prompt = f"""
    Generate a timeline-based answer for the following question by listing all entities, events, or attributes associated with the specified subject and time range. Ensure the output is in a structured, ordered format that reflects the temporal sequence accurately.

    You are given additional context about the query. Use the context as additional information to answer the query.

    Context:

    {combined_summaries}

    {few_shot_examples}

    Now, answer the following question:
    Question: {prompt_question}
    Answer:
    """

    generated_response = pipe(prompt.strip())[0]['generated_text']

    results.append({
        "question": prompt_question,
        "response": generated_response,
        "context": top_k_contexts,
        "examples": few_shot_examples
    })

results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv(f"../results/rag_top_{k_context}_few_shot_knn_{k_value}_xl.csv", index=False)